In [ ]:
import pandas as pd
import json
import time
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [ ]:
caminho_data = 'data_final.csv'
caminho_json = 'C:/Users/abner/Desktop/python/Polygon/dados_IBGE/Coordendas_municipios.geojson'
caminho_dados_mun = 'AR_BR_RG_UF_MES_MIC_MUN_2017.xlsx'

In [ ]:
dados_mun = pd.read_excel(caminho_dados_mun, encoding='latin-1')

In [ ]:
with open(caminho_json, encoding="latin-1") as file:
    coords_json = json.load(file)

In [ ]:
data_final = pd.DataFrame()

for i in range(0, len(coords_json['features'])):
    
    data_aux = pd.DataFrame(columns=['COD_UF','CD_GEOCMU','UF','ESTADO','CIDADE','COORDENADAS'])
    
    data_aux.loc[i,'COD_UF'] = coords_json['features'][i]['properties']['CD_GEOCMU'][0:2]
    data_aux.loc[i,'CD_GEOCMU'] = coords_json['features'][i]['properties']['CD_GEOCMU']
    
    data_aux.loc[i,'UF'] = dados_mun.iloc[ dados_mun.loc[dados_mun['CD_GCMUN'] == int(data_aux.loc[i,'CD_GEOCMU']) ].index[0]  ]['NM_UF_SIGLA'] 
    
    data_aux.loc[i,'ESTADO'] = dados_mun.iloc[ dados_mun.loc[dados_mun['CD_GCMUN'] == int(data_aux.loc[i,'CD_GEOCMU']) ].index[0]  ]['NM_UF'].upper()
    data_aux.loc[i,'CIDADE'] = dados_mun.iloc[ dados_mun.loc[dados_mun['CD_GCMUN'] == int(data_aux.loc[i,'CD_GEOCMU']) ].index[0]  ]['NM_MUN_2017'].upper()
    data_aux.loc[i,'COORDENADAS'] = str(coords_json['features'][i]['geometry']['coordinates']).replace('[[[','[[').replace(']]]',']]').replace('[[','[').replace(']]',']').replace(']','').replace('[','').replace(' ','') 
    
    data_final = data_final.append(data_aux)

In [132]:
for i,chunk in enumerate(pd.read_csv('data_final.csv', chunksize=1800,encoding='latin-1')):
    chunk.to_csv('chunk{}.csv'.format(i))

In [ ]:
data_final.to_csv('data_final.csv', encoding='latin-1')

In [ ]:
all_cities_coords = list()
cities_coords = list()
coords_total_mun =  list()

for i in range(0, data_final.shape[0]):
    coords_city = data_final['COORDENADAS'][i].split(',')
    city = list()
    for item in coords_city:
        city.append(float(item))
        
    all_cities_coords.append(city)
    print(str(100*i/data_final.shape[0]),end='\r')

k = 0 
while k <  len(all_cities_coords):
    p = 0
    cities_coords = list()
    while p < len(all_cities_coords[k]):
        x = (all_cities_coords[k][p],all_cities_coords[k][p+1] )
        p = p+2
        cities_coords.append(x)
    coords_total_mun.append(cities_coords)
    k = k+1

In [ ]:
#teste se está dentro do município informado
resultados = pd.DataFrame()
start_time = time.time()
for k in range(0, len(dados_mun)):
    index_coordenada_mun = data_final.loc[ data_final['CD_GEOCMU'] == str(dados_mun.CD_GCMUN[k]) ].index[0]
    
    polygon = Polygon(coords_total_mun[ index_coordenada_mun ])
    local = Point(float(dados_mun.LONG[k]), float(dados_mun.LAT[k]))
    
    if polygon.intersects(local) or polygon.contains(local):
        resultados = resultados.append([[dados_mun.CD_GCMUN[k],data_final.CD_GEOCMU[index_coordenada_mun]]])
    else:
        continue
print("--- %s seconds ---" % (time.time() - start_time))   

In [ ]:
resultados.to_excel('resultados.xlsx', encoding='latin-1')

In [125]:
#merge
k = 0
final_csv = pd.DataFrame()
while k <=17:
    final_csv = final_csv.append(pd.read_csv('chunk'+str(k)+'.csv',index_col=0))
    k=k+1
final_csv = final_csv.drop(final_csv.columns[0], axis=1)

In [126]:
final_csv.head()

,COD_UF,CD_GEOCMU,UF,ESTADO,CIDADE,COORDENADAS
0,15,1501956,PA,PARÁ,CACHOEIRA DO PIRIÁ,"-46.31955232,-1.755176174,-46.319895874,-1.755..."
1,21,2111250,MA,MARANHÃO,SÃO JOSÉ DOS BASÍLIOS,"-44.634092316,-5.034975275,-44.634069672,-5.03..."
2,52,5201454,GO,GOIÁS,APARECIDA DO RIO DOCE,"-51.142295175,-18.296724969,-51.142362302,-18...."
3,52,5202353,GO,GOIÁS,ARENÓPOLIS,"-51.699530442,-16.232696574,-51.699571438,-16...."
4,15,1502855,PA,PARÁ,CURUÁ,"-55.171927193,-1.833305029,-55.166468298,-1.73..."
